### Get the latest public contribution text file in MagIC

In [1]:
import requests

# Get the ID for the latest public MagIC contribution
contributions_response = requests.get(
    'https://api.earthref.org/v1/MagIC/search/contributions?size=1',
    headers={'Accept': 'application/json'}
)
latest_contribution_id = contributions_response.json()['results'][0]['id']

# Get the contribution text for the ID and print the first 1000 characters
latest_contribution_text = requests.get(
    'https://api.earthref.org/v1/MagIC/contribution/' + str(latest_contribution_id),
    headers={'Accept': 'text/plain'}
)
print(latest_contribution_text.content[0:1000])

b'tab delimited\tcontribution\nid\tversion\ttimestamp\tcontributor\tdata_model_version\treference\n16761\t2\t2020-02-23T23:03:45.034Z\t@ltauxe\t3.0\t10.1016/J.EPSL.2014.12.034\n>>>>>>>>>>\ntab delimited\tlocations\nlocation\tlocation_type\tcitations\texpedition_name\texpedition_ship\texpedition_leg\texpedition_url\tgeologic_classes\tlithologies\ttectonic_settings\tlat_s\tlat_n\tlon_w\tlon_e\tcontinent_ocean\tregion\tage_low\tage_high\tage_unit\nU1361A\tDrill Site\tThis study\t318\tJOIDES Resolution\t318\thttp://publications.iodp.org/proceedings/318/318toc.htm\tSedimentary\tSilicate Clay:Silty Clay\tContinental Shelf\t-64.2457\t-64.2457\t143.532\t143.532\tIndian Ocean\tWilkes Land Margin\t0\t12\tma\n>>>>>>>>>>\ntab delimited\tsites\nsite\tlocation\tresult_type\tresult_quality\tmethod_codes\tcitations\tgeologic_classes\tgeologic_types\tlithologies\tlat\tlon\tcore_depth\tage_low\tage_high\tage_unit\texternal_results\n318-U1361A-001H-2-W-35\tU1361A\ti\tg\tSO-V:FS-C-DRILL-IODP\tThis study\t

### Download a public contribution text file in MagIC by DOI

In [2]:
import requests

# Download the contribution text for the DOI and print the first 1000 characters
contribution_text = requests.post(
    'https://api.earthref.org/v1/MagIC/download',
    data={'doi': '10.1016/J.EPSL.2014.12.034'},
    headers={'Accept': 'text/plain'}
)
print(contribution_text.content[0:1000])

b'tab delimited\tcontribution\nid\tversion\ttimestamp\tcontributor\tdata_model_version\treference\n16761\t2\t2020-02-23T23:03:45.034Z\t@ltauxe\t3.0\t10.1016/J.EPSL.2014.12.034\n>>>>>>>>>>\ntab delimited\tlocations\nlocation\tlocation_type\tcitations\texpedition_name\texpedition_ship\texpedition_leg\texpedition_url\tgeologic_classes\tlithologies\ttectonic_settings\tlat_s\tlat_n\tlon_w\tlon_e\tcontinent_ocean\tregion\tage_low\tage_high\tage_unit\nU1361A\tDrill Site\tThis study\t318\tJOIDES Resolution\t318\thttp://publications.iodp.org/proceedings/318/318toc.htm\tSedimentary\tSilicate Clay:Silty Clay\tContinental Shelf\t-64.2457\t-64.2457\t143.532\t143.532\tIndian Ocean\tWilkes Land Margin\t0\t12\tma\n>>>>>>>>>>\ntab delimited\tsites\nsite\tlocation\tresult_type\tresult_quality\tmethod_codes\tcitations\tgeologic_classes\tgeologic_types\tlithologies\tlat\tlon\tcore_depth\tage_low\tage_high\tage_unit\texternal_results\n318-U1361A-001H-2-W-35\tU1361A\ti\tg\tSO-V:FS-C-DRILL-IODP\tThis study\t

### Get the 50 latest public sites in MagIC that mention "devonian"

In [3]:
import requests
import pandas

# Get the 50 latest sites in MagIC
response = requests.get(
    'https://api.earthref.org/v1/MagIC/search/sites',
    params={'query': 'devonian', 'size': 50},
    headers={'Accept': 'application/json'}
)

# Check the response status code
if (response.status_code == 404):
  print('Request URL is incorrect.')

# Parse the JSON output into a dictionary
json_response = response.json()

# Retrieve the list of sites, of which each site may have multiple data rows
sites = json_response['results']

# Flatten the sites rows
def flatten(listOflists):
    return [item for sublist in listOflists for item in sublist]
sites_rows = flatten(sites)

sites_df = pandas.DataFrame(sites_rows)
sites_df.head()

,vgp_lat,lon,geologic_types,formation,dir_k,method_codes,geologic_classes,vgp_lon,vgp_dm,citations,...,dir_tilt_correction,dir_nrm_origin,result_quality,age,description,age_sigma,int_abs,external_database_ids,dir_r,int_abs_sigma
0,-29.2,109.28,sediment layer,Yuntaiguan Formation,100.6,DE-BFL-A,sedimentary,49.6,6.8,10.1029/2019GL083123,...,100,p,g,385,NaN,NaN,NaN,NaN,NaN,NaN
1,-31.1,109.28,sediment layer,Yuntaiguan Formation,32,DE-BFL-A,sedimentary,52.3,11,10.1029/2019GL083123,...,100,p,g,385,NaN,NaN,NaN,NaN,NaN,NaN
2,-28.9,109.28,sediment layer,Yuntaiguan Formation,34.5,DE-BFL-A,sedimentary,48.2,11.8,10.1029/2019GL083123,...,100,p,g,385,NaN,NaN,NaN,NaN,NaN,NaN
3,-36,109.28,sediment layer,Yuntaiguan Formation,94.3,DE-BFL-A,sedimentary,54.5,7,10.1029/2019GL083123,...,100,p,g,385,NaN,NaN,NaN,NaN,NaN,NaN
4,-16.4,109.28,sediment layer,Yuntaiguan Formation,1486,DE-BFL-A,sedimentary,41.7,1.8,10.1029/2019GL083123,...,100,p,g,385,NaN,NaN,NaN,NaN,NaN,NaN
